---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
#####!!REVIEW!!##
# ネットワークのモデリング
# ◆ Preferential Attachment Model◆
#      ネットワーク上のDegreeの内容から将来どう派生するかをModelingする
#   　　　特徴: degreeが多いノードほど数が少なく、ほどんどのノードはdegreeが1,2など = Power-low degree destribution
#      理由: 有名なactorは映画に引っ張りだこなので新しく入った他のactorと結びつく可能性が高いなどの理由。
#                Clustering Coefficientが小さい。(shortest pathも小さい)
#
#     P(k) ... degree distributionを求める式 k個のdegreeを持つノードの数 / 全ノード数
#     Pの最も大きくなる k とは ... histogram上で一番多いdegree
#
#         degrees = G.degree() # key: node名, value: degree
#         degree_values = sorted(set(degrees.values())) # degree情報だけをソート
#         # Histogramを求める(degreeの小さいものから分布をリストに格納する)
#         histogram = [ list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_values ]
#
#         import matplotlib.pyplot as plt
#         plt.bar(degree_values, histogram)
#         plt.xlabel('Degree')
#         plt.ylabel('Fraction of Nodes') # どれぐらいあるか(分布しているか)
#         plt.show()
#         
#         - node数で割っているので、少数になる(現実世界ではデータ多いのでlogで表示する（下ほど稀）)
#          - In-Degreeで求めたい時は G.in_degree()を使用する。
#            ⅰ. ２つのノードと1つのedgeから始まる
#            ⅱ. ノードを１つのedgeと一緒に追加していく（ランダムにだが、probabilityはdegreeの数に比例する）　◆◆特徴◆◆
#    Preferential Attachment Modelの代表的なメソッド: barabashi_albert_graph(n,m)   n: 新しいノード、m: 既存のノード、 結びついたグラフを返す
#         G = nx.barabashi_albert_graph(1000000, 1) # 1000000の新しいノードを1つの既存のノードに結び付けさせる
#         degrees = G.degree()
#         degree_values = sorted(set(degrees.values())) # degree情報だけをソート
#         # Histogramを求める(degreeの小さいものから分布をリストに格納する)
#         histogram = [ list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_values ]
#
#         plt.bar(degree_values, histogram, 'o')
#         plt.xlabel('Degree')
#         plt.ylabel('Fraction of Nodes')
#         plt.xscale('log')
#         plt.yscale('log')
#         plt.show()
#         
# ◆ Small World Model◆
#      Path Length and ClusteringをModelingする
#   　　　特徴: Real NetworkはClustering Coefficientが高い傾向があり、平均path-lengthが小さい。
#      何故か: 限られた少数のノードが多くのdegreeを持つので、そのノードがハブとなり(ハブが多いほど)、clustering coefficientが大きく、平均path-lengthが小さくなるから。
#                pが小さい時点ではClustering Coefficientが高くshortest pathは小さい。
#   　　　k: nearest neighborの数(edgeで繋がる)、p: ランダムなノードとrewireを行うprobability(通常かなり小さい)
#      pが小さくrewireがない状態だとLocal Clustering Coefficientは大きい。pが大きくなると今度は平均path-lengthが小さくなる。
#      pが0から1の間ではaverage shortest pathが急速に減り、average clustering coefficientがゆっくり減る。◆◆特徴◆◆
#
#         G = nx.barabashi_albert_graph(1000, 4) # 1000の新しいノードを4つの既存のノードに結び付けさせる
#         print(nx.average_clustering(G))
#         => 0.0202859
#         print(nx.average_shortest_path_length(G))
#         => 4.1694294
#         
#    Small World Modelの代表的なメソッド: watts_strogatz_graph(n,k,p)   n: ノード数
#         G = nx.watts_strogatz_graph(1000, 6, 0.04) #  p=0.04は大きすぎるような..
#         degrees = G.degree()
#         degree_values = sorted(set(degrees.values())) # degree情報だけをソート
#         # Histogramを求める(degreeの小さいものから分布をリストに格納する)
#         histogram = [ list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_values ]
#         plt.bar(degree_values, histogram, 'o')
#         plt.xlabel('Degree')
#         plt.ylabel('Fraction of Nodes')   <====== ほとんどのノードがDegree=6になる。(理由:probabilityで一律に繋ぎ直しているから)
#         plt.show()
#
# ◆ Link Prediction◆
#      代表利用例：フレンド・レコメンド システム
#      (◆特徴◆ネットワーク上のどのノードがノードAと結びつきそうかを推測する)
#      ネットワーク上のに次どのようなedgeが構成されそうかをModelingする
#   　　　特徴: Real NetworkはClustering Coefficientが高い傾向があり、平均path-lengthが小さい。
#
#       Triadic closure: 人がソーシャルネットワークの中でコネクションを築く傾向
#       
#      5 basic measures
#        1. 共通の友人が多い
#         - Common Neighbors  ... comm_neigh(X,Y) = | N(X) かつ N(Y) | 、N(X) : XのNeighbors(edgeで繋がっている)
#        2. (共通の友人 / 友人の数) が多い
#         - Jaccard Coefficient  ... jacc_coedd(X,Y) = comm_neigh(X,Y)  ÷  | N(X) 又は N(Y) |  (重複カウントはしない。)
#        3. 共通の友人の友人数が少ない友人が沢山いる
#         - Resource Allocation Index  ... 共通のNeighborが多く、そのNeighborのdegreeが小さいほど値が大きくなるIndex
#               共通のNeighborがZとしてそのZのdegreeが４の場合、1/4となる。これを全ての共通Neighborに対し求めsumする。
#         - Adamic-Adar Index ... Resource Allocation Index のpenalize方法をlog()に変えただけ(degreeが3ならlog(3)で割る)
#        4. 友人がどっちもとても多い(ひょっとして友人では?)
#         - Preferential Attachment Score ... pref_attach(X,Y) = | N(X)| * | N(Y) |  、各々のdegreeを沢山持っていたらより繋がりやすい。
#       
#　　　　　　　　　# 全ノードのcommon neighbor(お互いedgeで繋がりのあるノードの数)を求める式
#         common_neigh = [ (e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1])))) for e in nx.non_edges(G) ]
#         sorted(common_neigh, key=operator.itemgetter(2), reverse=True)
#         print(common_neigh) 
#       
#　　　　　　　　　# 全ノードのcommon neighborを全てのneighborsで割ってノーマライズしたものを求める式
#         L = list(nx.jaccard_coefficient(G))
#         L.sort(key=operator.itemgetter(2), reverse=True)
#         print(L)
#       
#　　　　　　　　　# 全ノードのcommon neighborを共通のneighborsのdegree数でピーナライズした(割る)もののsum()を求める式
#         L = list(nx.resource_allocation_index(G))
#         L.sort(key=operator.itemgetter(2), reverse=True)
#         print(L)
#       
#　　　　　　　　　# 全ノードのdegreeを元に計算する。値は整数(pref_attach(A,B)の値を求めるならAのdegree × Bのdegreeなので)
#         L = list(nx.preferential_attachment(G))
#         L.sort(key=operator.itemgetter(2), reverse=True)
#         print(L)
#
#    Community
#         2つのノード間のSoundarajan-Hopcroft score
#          = | N(X) かつ N(Y) | + 共通の友人の数だけsum(f(u))
#          ここで f(u) = { 1, u in same comm. as X and Y     0, otherwise}
#         同じコミュニティの場合、ボーナスする
#         | N(X) かつ N(Y) |  が全員同じコミュニティなら | N(X) かつ N(Y) | をもう一回足す。
#       ２つのメジャーな式
#         - Common Neighbors Soundarajan-Hopcroft score
#         - Resource Allocation Soundarajan-Hopcroft score
###############

import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [ ]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [ ]:
def graph_identification():
    
    # Your Code Here
    
    return # Your Answer Here

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [ ]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [ ]:
def salary_predictions():
    
    # Your Code Here
    
    return # Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [3]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    
    # Your Code Here
    
    return # Your Answer Here